In [22]:
# Step 1: Import necessary libraries
import pandas as pd
from datetime import datetime

# Step 2: Load the data from CSV files
product_df = pd.read_csv("C:/Users/Vaibhav/Desktop/inotask3/diminos_data_v2/products.csv")
orders_df = pd.read_csv("C:/Users/Vaibhav/Desktop/inotask3/diminos_data_v2/orders.csv")
order_items_df = pd.read_csv("C:/Users/Vaibhav/Desktop/inotask3/diminos_data_v2/order_items.csv")
deliveries_df = pd.read_csv("C:/Users/Vaibhav/Desktop/inotask3/diminos_data_v2/deliveries.csv")

# Step 3: Filter deliveries dataframe for "Delivered" status
delivered_deliveries = deliveries_df[deliveries_df['status'] == "Delivered"]

# Step 4: Merge necessary dataframes
merged_df = pd.merge(orders_df, order_items_df, on="order_id")
merged_df = pd.merge(merged_df, delivered_deliveries, on="order_id", how="left")  # Use left join to handle missing "Delivered" statuses

# Step 5: Convert timestamps to datetime objects
merged_df['order_placed_at'] = pd.to_datetime(merged_df['order_placed_at'])
merged_df['time_stamp'] = pd.to_datetime(merged_df['time_stamp'])

# Step 6: Calculate delivery time only for orders with "Delivered" status and with valid delivery timestamps
merged_df['delivery_time'] = 0  # Initialize delivery time column with zeros

for index, row in merged_df.iterrows():
    if row['status'] == "Delivered" and not pd.isnull(row['time_stamp']):  # Check if status is "Delivered" and delivery timestamp is not null
        merged_df.at[index, 'delivery_time'] = (row['time_stamp'] - row['order_placed_at']).total_seconds() / 60
        
# Step 7: Handle NaN values in delivery_time column
merged_df['delivery_time'] = merged_df['delivery_time'].fillna(0)  # Replace NaN with 0 for orders without "Delivered" status

# Step 8: Extract item IDs from the 'items_ordered' column and explode them into separate rows
merged_df['item_id'] = merged_df['items_ordered'].str.extract(r"p_(\d+)")
merged_df = merged_df.explode('item_id')

# Step 9: Add prefix "p_" to the item_id column in merged_df dataframe
merged_df['item_id'] = 'p_' + merged_df['item_id']

# Step 10: Check if 'Price' column exists in the DataFrame
if 'Price' in product_df.columns:
    # Step 11: Merge with product_df to get price information
    merged_df = pd.merge(merged_df, product_df[['item_id', 'Price']], on='item_id', how='left')

    # Step 12: Calculate refund amount for orders exceeding 30 minutes delivery time
    merged_df['refund_amount'] = merged_df['Price'].where(merged_df['delivery_time'] > 30, 0)

    # Step 13: Calculate average delivery time
    average_delivery_time = merged_df['delivery_time'].mean()
    print("Average Delivery Time:", round(average_delivery_time, 2), "minutes")

    # Step 14: Calculate 99th percentile delivery time
    percentile_99_delivery_time = merged_df['delivery_time'].quantile(0.99)
    print("99th Percentile Delivery Time:", round(percentile_99_delivery_time, 2), "minutes")

    # Step 15: Determine which metric is better for evaluating delivery time
    print("Metric for Delivery Time Evaluation: 99th Percentile")

    # Step 16: Find order with maximum delivery time
    max_delivery_order_id = merged_df.loc[merged_df['delivery_time'].idxmax()]['order_id']
    print("Order ID with Maximum Delivery Time:", max_delivery_order_id)


    # Step 17: Filter orders between 1st January and 31st January 2024
    filtered_orders = merged_df[(merged_df['order_placed_at'] >= datetime(2024, 1, 1)) & 
                                (merged_df['order_placed_at'] <= datetime(2024, 1, 31))]
    print("Number of pizzas ordered between 1st January and 31st January 2024:", len(filtered_orders))

    # Step 18: Count orders that took more than 30 minutes to deliver between 1st January and 31st January 2024
    late_deliveries = filtered_orders[filtered_orders['delivery_time'] > 30]
    print("Number of orders took more than 30 minutes to deliver between 1st January and 31st January 2024:", len(late_deliveries))

    # Step 19: Calculate total refund amount in 2023
    refund_amount_2023 = merged_df[(merged_df['order_placed_at'].dt.year == 2023)]['refund_amount'].sum()
    print("Total refund amount in 2023:", refund_amount_2023)

    # Step 20: Determine year with maximum refund amount
    refund_amount_by_year = merged_df.groupby(merged_df['order_placed_at'].dt.year)['refund_amount'].sum()
    max_refund_year = refund_amount_by_year.idxmax()
    print("Year with maximum refund amount:", max_refund_year)
    
    # Step 21: Calculate total revenue generated by the pizza store (excluding refunds)
    total_revenue = merged_df['Price'].sum() - merged_df['refund_amount'].sum()
    print("Total revenue generated by the pizza store (excluding refunds):", total_revenue)

    # Step 22: Calculate total orders count
    total_orders_count = len(merged_df['order_id'].unique())
    print("Total orders count:", total_orders_count)

    # Step 23: Calculate total count of orders refunded
    total_refunded_orders_count = len(merged_df[merged_df['refund_amount'] > 0]['order_id'].unique())
    print("Total count of orders refunded:", total_refunded_orders_count)

else:
    print("Price column does not exist in the DataFrame.")


Average Delivery Time: 20.65 minutes
99th Percentile Delivery Time: 30.9 minutes
Metric for Delivery Time Evaluation: 99th Percentile
Order ID with Maximum Delivery Time: 321629049
Number of pizzas ordered between 1st January and 31st January 2024: 11894
Number of orders took more than 30 minutes to deliver between 1st January and 31st January 2024: 158
Total refund amount in 2023: 1003577
Year with maximum refund amount: 2023
Total revenue generated by the pizza store (excluding refunds): 220578761
Total orders count: 454336
Total count of orders refunded: 6119


In [17]:
# Count the total number of orders refunded
total_refunded_orders = merged_df[merged_df['refund_amount'] > 0]['order_id'].nunique()

print("Total count of orders refunded:", total_refunded_orders)


Total count of orders refunded: 6119


In [18]:
# Count the total number of orders
total_orders_count = merged_df['order_id'].nunique()

print("Total orders count:", total_orders_count)


Total orders count: 454336


In [21]:
print("Delivery time for order ID 321629049:", merged_df.loc[merged_df['order_id'] == 321629049]['delivery_time'].values[0], "minutes")


Delivery time for order ID 321629049: 62.91207811666666 minutes


In [7]:
print(product_df.columns)


Index(['item_id', 'category', 'Item', 'Size', 'Price'], dtype='object')
